In [ ]:
!pip install jupyterlab_execute_time

In [ ]:
!pip3 install --upgrade pip

In [ ]:
!pip install distributed

In [ ]:
!python3 -m pip install "dask[complete]" --upgrade

In [ ]:
!pip install bokeh

In [17]:
!pip install lucidity

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 13.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492025 sha256=94ac60794ddfa1f8c13d6282435d8b4cd7b1455998a84dcf46709a12b54ada75
  Stored in directory: /home/acmbrito/.cache/pip/wheels/a0/0b/ee/e6994fadb42c1354dcccb139b0bf2795271bddfe6253ccdf11
Successfully built future
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is

In [189]:
import io
import tqdm
import glob
import json
import pandas as pd
import dask.dataframe as dd

In [190]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=4, processes=True, memory_limit='8GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33831,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 29.80 GiB
Comm: tcp://127.0.0.1:37285,Total threads: 4
Dashboard: http://127.0.0.1:39257/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:46797,


In [204]:
client.shutdown()

In [191]:
meta = {
    'id': int,
    'created_at': object,
    'in_reply_to_id': float,
    'in_reply_to_account_id': float,
    'sensitive': bool,
    'spoiler_text': object,
    'visibility': object,
    'language': object,
    'uri': object,
    'url': object,
    'replies_count': int,
    'reblogs_count': int,
    'favourites_count': int,
    'edited_at': object,
    'content': object,
    'reblog': object,
    'application': object,
    'account': object,
    'media_attachments': object,
    'mentions': object,
    'tags': object,
}

# 'id', 'created_at', 'in_reply_to_id', 'in_reply_to_account_id',
#        'sensitive', 'spoiler_text', 'visibility', 'language', 'uri', 'url',
#        'replies_count', 'reblogs_count', 'favourites_count', 'edited_at',
#        'content', 'reblog', 'application', 'account', 'media_attachments',
#        'mentions', 'tags', 'emojis', 'card', 'poll'

In [192]:
import dask.bag as db
from pandas.core.dtypes.common import pandas_dtype as dtype

def read_data(filename):
    print(filename)
    try:
        # try reading in the data with pandas
        df = pd.read_json(filename, orient='values')
        to_drop = []
        for col in df.columns:
            if col not in meta:
                to_drop.append(col)

        df = df.drop(columns=to_drop)
        if 'created_at' in df.columns:
            df['created_at'] = df['created_at'].astype(object)
        if 'edited_at' in df.columns:
            df['edited_at'] = df['edited_at'].astype(object)

        columns = list(meta.keys())
        df = df[columns]
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df_current = pd.read_json(filename, orient='values')
        print('current columns', df_current.columns)
        df = pd.read_csv(io.StringIO(''), names=meta.keys(), dtype=meta)
    return df

In [193]:
files = glob.glob('posts_following/*')
print(len(files))
read_data(files[0]).head(3)

817
posts_following/posts_all_users_following_scholars_2023-09-21 16:44:25.903218.json


,id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,reblogs_count,favourites_count,edited_at,content,reblog,application,account,media_attachments,mentions,tags
0,109626125424847024,2023-01-03 16:05:23.600000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109589349151326505', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]
1,109626125378539168,2023-01-03 16:05:22.902000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109589349196868897', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]
2,109603230696844752,2022-12-30 15:02:57.774000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109596145488156680', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]


In [194]:
from dask import delayed
read_data_dd = delayed(read_data)

In [195]:
posts = read_data_dd(files[0]) #('posts_following/*')  

In [196]:
posts.head(2).compute()

,id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,reblogs_count,favourites_count,edited_at,content,reblog,application,account,media_attachments,mentions,tags
0,109626125424847024,2023-01-03 16:05:23.600000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109589349151326505', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]
1,109626125378539168,2023-01-03 16:05:22.902000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109589349196868897', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]


In [197]:
posts.columns

Delayed('getattr-295bd74e97ed5b059bcf749c13023109')

In [198]:
# columns = list(meta.keys())

# for file in tqdm.tqdm(files, total=len(files)):
#     temp = read_data_dd(file).compute()
#     if tuple(temp.columns) != tuple(columns):
#         print(file)
#         print(temp.columns)
#         print()
#         break

In [199]:
df = [read_data_dd(file) for file in files]
posts = dd.from_delayed(df, meta=meta)

In [200]:
posts.head(3)

,id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,reblogs_count,favourites_count,edited_at,content,reblog,application,account,media_attachments,mentions,tags
0,109626125424847024,2023-01-03 16:05:23.600000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109589349151326505', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]
1,109626125378539168,2023-01-03 16:05:22.902000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109589349196868897', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]
2,109603230696844752,2022-12-30 15:02:57.774000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109596145488156680', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]


In [201]:
posts.head(3)

,id,created_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,url,...,reblogs_count,favourites_count,edited_at,content,reblog,application,account,media_attachments,mentions,tags
0,109626125424847024,2023-01-03 16:05:23.600000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109589349151326505', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]
1,109626125378539168,2023-01-03 16:05:22.902000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109589349196868897', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]
2,109603230696844752,2022-12-30 15:02:57.774000+00:00,NaN,NaN,False,,public,None,https://mstdn.science/users/kirt/statuses/1096...,https://mstdn.science/users/kirt/statuses/1096...,...,0,0,NaT,,"{'id': '109596145488156680', 'created_at': '20...",None,"{'id': '109321353559113983', 'username': 'kirt...",[],[],[]


In [202]:
import lucidity

def get_user_server(uri):
    try:
        template = lucidity.Template('model', 'https://{server_name}/users/{user_name}/statuses/{post_id}')
        parsed = template.parse(uri)
        return "{}@{}".format(parsed['user_name'], parsed['server_name'])
    except:
        print('deu ruim', uri)
        return ""

unique_users = posts.uri.apply(get_user_server, meta=('user', str)).unique()

In [207]:
unique_users.to_csv('unique_users_saved_16_10', sep='\t')

posts_following/posts_all_users_following_scholars_2023-09-21 16:44:25.903218.json
posts_following/posts_all_users_following_scholars_2023-09-21 17:01:34.935625.json
posts_following/posts_all_users_following_scholars_2023-09-21 17:08:16.183161.json
posts_following/posts_all_users_following_scholars_2023-09-22 13:09:25.327896.json
posts_following/posts_all_users_following_scholars_2023-09-22 13:43:49.063406.json
posts_following/posts_all_users_following_scholars_2023-09-22 14:03:14.084235.json
posts_following/posts_all_users_following_scholars_2023-09-22 14:16:15.462544.json
posts_following/posts_all_users_following_scholars_2023-09-22 14:24:59.825458.json
posts_following/posts_all_users_following_scholars_2023-09-22 14:29:44.958438.json
posts_following/posts_all_users_following_scholars_2023-09-22 14:35:41.283337.json
posts_following/posts_all_users_following_scholars_2023-09-22 14:40:58.700196.json
posts_following/posts_all_users_following_scholars_2023-09-22 14:46:33.561099.json
post

['/mnt/e/Ana/mastodon-proj/notebooks/mastodon_api/unique_users_saved_16_10/0.part']